## shot.py

In [ ]:
import time
import cv2
def shot(IMAGE):
    camera_port = 0
    camera = cv2.VideoCapture(camera_port)
    time.sleep(0.1)  # If you don't wait, the image will be dark
    return_value, image = camera.read()
    cv2.imwrite(IMAGE, image)
    del(camera)

## train.py

In [ ]:
from keras.applications import MobileNet
from keras.models import Sequential,Model 
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

# MobileNet is designed to work with images of dim 224,224
img_rows,img_cols = 224,224

MobileNet = MobileNet(weights='imagenet',include_top=False,input_shape=(img_rows,img_cols,3))

# Here we freeze the last 4 layers
# Layers are set to trainable as True by default

for layer in MobileNet.layers:
    layer.trainable = True

# Let's print our layers
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i),layer.__class__.__name__,layer.trainable)

def addTopModelMobileNet(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(512,activation='relu')(top_model)
    
    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model

num_classes = 5

FC_Head = addTopModelMobileNet(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

train_data_dir = '/Users/durgeshthakur/Deep Learning Stuff/Emotion Classification/fer2013/train'
validation_data_dir = '/Users/durgeshthakur/Deep Learning Stuff/Emotion Classification/fer2013/validation'

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=30,
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols),
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )

validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')

from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

checkpoint = ModelCheckpoint(
                             'emotion_face_mobilNet.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          verbose=1,restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

nb_train_samples = 24176
nb_validation_samples = 3006

epochs = 25

history = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size)

## test.py

In [1]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
classifier =load_model('./Emotion_Detection.h5')

class_labels = ['Angry','Happy','Neutral','Sad','Surprise']

cap = cv2.VideoCapture(0)



while True:
    # Grab a single frame of video
    ret, frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)


        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

        # make a prediction on the ROI, then lookup the class

            preds = classifier.predict(roi)[0]
            print("\nprediction = ",preds)
            label=class_labels[preds.argmax()]
            print("\nprediction max = ",preds.argmax())
            print("\nlabel = ",label)
            label_position = (x,y)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
        else:
            cv2.putText(frame,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
        print("\n\n")
    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


prediction =  [0.06370778 0.00867059 0.8161519  0.09966525 0.01180443]

prediction max =  2

label =  Neutral




prediction =  [0.05104404 0.02572583 0.81977385 0.08560906 0.01784729]

prediction max =  2

label =  Neutral




prediction =  [0.04430351 0.03465727 0.8252957  0.07460175 0.02114185]

prediction max =  2

label =  Neutral




prediction =  [0.04512878 0.01829038 0.8398001  0.07500352 0.02177728]

prediction max =  2

label =  Neutral




prediction =  [0.03751355 0.0132869  0.88403946 0.05546831 0.00969183]

prediction max =  2

label =  Neutral




prediction =  [0.03542163 0.00835109 0.8948781  0.05097267 0.01037659]

prediction max =  2

label =  Neutral




prediction =  [0.03563857 0.00647501 0.89733124 0.04915987 0.01139531]

prediction max =  2

label =  Neutral




prediction =  [0.03952926 0.00625356 0.88242227 0.0574231  0.01437179]

prediction max =  2

label =  Neutral




prediction =  [0.03180299 0.00709969 0.9063067  0.04460336 0.01018736]

prediction max 


prediction =  [0.15050592 0.00282754 0.6669374  0.16868576 0.01104341]

prediction max =  2

label =  Neutral




prediction =  [0.12005249 0.00395429 0.70807844 0.15799245 0.00992232]

prediction max =  2

label =  Neutral




prediction =  [0.09221391 0.00472672 0.77671254 0.11684442 0.0095024 ]

prediction max =  2

label =  Neutral




prediction =  [0.09337503 0.0038196  0.76051784 0.13154502 0.01074259]

prediction max =  2

label =  Neutral




prediction =  [0.18362185 0.00245826 0.5985486  0.20722677 0.00814447]

prediction max =  2

label =  Neutral




prediction =  [0.08130657 0.00507378 0.80041564 0.10550833 0.00769569]

prediction max =  2

label =  Neutral




prediction =  [0.11629265 0.003094   0.7219377  0.14747159 0.01120401]

prediction max =  2

label =  Neutral




prediction =  [0.13985243 0.00315277 0.6535477  0.18431701 0.01912997]

prediction max =  2

label =  Neutral




prediction =  [0.09772889 0.00334684 0.7633862  0.12128585 0.01425222]

prediction max 



prediction =  [0.05833789 0.0083146  0.8178122  0.08533992 0.03019538]

prediction max =  2

label =  Neutral




prediction =  [0.05828669 0.00578407 0.80923283 0.08551788 0.0411786 ]

prediction max =  2

label =  Neutral




prediction =  [0.04748859 0.00606186 0.85661745 0.06978421 0.02004787]

prediction max =  2

label =  Neutral




prediction =  [0.05190596 0.00924865 0.8250537  0.08129346 0.03249821]

prediction max =  2

label =  Neutral




prediction =  [0.06005872 0.00829421 0.7965382  0.09684547 0.03826347]

prediction max =  2

label =  Neutral




prediction =  [0.052558   0.00943098 0.8385999  0.08266115 0.01674987]

prediction max =  2

label =  Neutral




prediction =  [0.06622908 0.00910243 0.7896061  0.10673741 0.02832495]

prediction max =  2

label =  Neutral




prediction =  [0.05538025 0.0066778  0.8297988  0.0886298  0.01951327]

prediction max =  2

label =  Neutral




prediction =  [0.06449661 0.01103767 0.785975   0.11033428 0.02815645]

prediction max

label =  Neutral




prediction =  [0.05403024 0.0057408  0.8704942  0.06559635 0.00413841]

prediction max =  2

label =  Neutral




prediction =  [0.04841676 0.00531042 0.8834596  0.05815036 0.00466289]

prediction max =  2

label =  Neutral




prediction =  [0.05029447 0.00555558 0.8789176  0.06150058 0.00373176]

prediction max =  2

label =  Neutral




prediction =  [0.03723915 0.00715646 0.9036924  0.04824012 0.00367195]

prediction max =  2

label =  Neutral




prediction =  [0.03723915 0.00715646 0.9036924  0.04824012 0.00367195]

prediction max =  2

label =  Neutral




prediction =  [0.04790521 0.00673151 0.877047   0.06227245 0.0060438 ]

prediction max =  2

label =  Neutral




prediction =  [0.07437212 0.00351858 0.8222456  0.09411865 0.00574506]

prediction max =  2

label =  Neutral




prediction =  [0.12784694 0.00253678 0.69091976 0.16893803 0.00975848]

prediction max =  2

label =  Neutral




prediction =  [0.1582551  0.00232794 0.62207484 0.20765373 0.009688


prediction =  [0.05728206 0.00519388 0.8461822  0.08432306 0.00701873]

prediction max =  2

label =  Neutral




prediction =  [0.06149713 0.00528153 0.8258496  0.098681   0.00869064]

prediction max =  2

label =  Neutral




prediction =  [0.05260966 0.00389771 0.8587771  0.07845693 0.00625851]

prediction max =  2

label =  Neutral




prediction =  [0.05807919 0.00466275 0.84253985 0.08544756 0.00927064]

prediction max =  2

label =  Neutral




prediction =  [0.04837748 0.00489493 0.8682596  0.07153913 0.00692876]

prediction max =  2

label =  Neutral




prediction =  [0.05990979 0.00518488 0.8354787  0.09234819 0.00707843]

prediction max =  2

label =  Neutral




prediction =  [0.07219418 0.00450296 0.79009646 0.11832644 0.01488003]

prediction max =  2

label =  Neutral




prediction =  [0.06425641 0.0029547  0.8277453  0.09738422 0.00765926]

prediction max =  2

label =  Neutral




prediction =  [0.06288322 0.00402746 0.8245747  0.10127176 0.00724296]

prediction max 

prediction =  [0.08759059 0.01078031 0.75539774 0.13346757 0.01276373]

prediction max =  2

label =  Neutral




prediction =  [0.14795828 0.01576222 0.59083456 0.23338942 0.01205547]

prediction max =  2

label =  Neutral




prediction =  [0.17081103 0.01442084 0.51626205 0.28934348 0.00916267]

prediction max =  2

label =  Neutral




prediction =  [0.1583007  0.0185822  0.5475418  0.26322892 0.01234644]

prediction max =  2

label =  Neutral




prediction =  [0.12888041 0.02666315 0.61812454 0.2129862  0.01334558]

prediction max =  2

label =  Neutral




prediction =  [0.13533433 0.01916114 0.6195515  0.21347526 0.01247781]

prediction max =  2

label =  Neutral




prediction =  [0.17777972 0.01829538 0.5224798  0.2711084  0.01033671]

prediction max =  2

label =  Neutral




prediction =  [0.15912795 0.02239246 0.5293562  0.28167495 0.00744851]

prediction max =  2

label =  Neutral




prediction =  [0.15096346 0.01774121 0.5907423  0.22886834 0.01168472]

prediction max =


prediction =  [0.09024508 0.01018501 0.7559997  0.13834527 0.00522489]

prediction max =  2

label =  Neutral





## emotion_func.py

In [ ]:
import cv2
import numpy as np
import os 
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
from shot import shot

# recognizer = cv2.face.LBPHFaceRecognizer_create()
# recognizer.read('trainer/trainer.yml')
def emotion():
    shot('opencv.png')
    classifier =load_model('./Emotion_Detection.h5')
    cascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascadePath);

    font = cv2.FONT_HERSHEY_SIMPLEX

    #iniciate id counter
    id = 0

    # Emotions related to ids: example ==> Anger: id=0,  etc
    names = ['Angry','Happy','Neutral','Sad','Surprise']

    img = cv2.imread("opencv.png")


    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        )

    for(x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)
        if np.sum([roi_gray])!=0:
                roi = roi_gray.astype('float')/255.0
                roi = img_to_array(roi)
                roi = np.expand_dims(roi,axis=0)

                preds = classifier.predict(roi)[0]
                print("\nprediction = ",preds)
                label=names[preds.argmax()]
                print("\nprediction max = ",preds.argmax())
                print("\nlabel = ",label)
                label_position = (x,y)
                cv2.putText(img,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
                cv2.imwrite("opencv_test.png",img)
                print("\n [INFO] Done detecting and Image is saved")
                return label 
        else :
            print('not found')